### Objective:

* loading point shapefiles to postgis:
    - city rank
    - model 1 & its Accuracy
    - model 2 & its Accuracy
    - model 3 & its Accuracy

In [7]:
import psycopg2
import arcpy

In [60]:
#################################################### LOADING MODEL 1: ALPHA = 2 ####################################################
##load shapefile to PostGIS: use pyscopg2

#connect to cloud database using psycopg2
#conn = psycopg2.connect(host='34.123.61.225', database='lab0', user='postgres', password='')
conn = psycopg2.connect(host='spatialdb.gisandbox.org', database='marso093', user='marso093')

#create cursor object for interacting with PostGIS
pgcursor = conn.cursor()
print('creating table...')

pgcursor.execute("DROP TABLE IF EXISTS MN_Cities_Pts_WGS_norank;")

pgcursor.execute("CREATE TABLE MN_Cities_Pts_WGS_norank(gid serial primary key, shape geometry(POINT, 4326), CITY_SJ text, Presence double precision, Accuracy double precision, overall_FP double precision, overall_TN double precision, overall_TP double precision, overall_FN double precision);")

print('Populating Table...')
cur = arcpy.da.SearchCursor('MN_Cities_Pts_WGS', ('SHAPE@WKT','CITY_SJ', 'Presence', 'Accuracy', 'overall_FP', 'overall_TN', 'overall_TP', 'overall_FN'))
count = 0
for row in cur:
    count +=1
    print(count, row)
    geomwkt = row[0]
    city_names = row[1]
    pres = row[2]
    acc = row[3]
    fp = row[4]
    tn = row[5]
    tp = row[6]
    fn = row[7]
    pgcursor.execute ("INSERT into MN_Cities_Pts_WGS_norank (shape, CITY_SJ, Presence, Accuracy, overall_FP, overall_TN, overall_TP, overall_FN) VALUES(ST_GeomFromText(%s, 4326), %s, %s, %s, %s, %s, %s, %s)", (geomwkt, city_names, pres, acc, fp, tn, tp, fn))

del cur
#commit changes and close connection
conn.commit()
conn.close()

creating table...
Populating Table...
1 ('POINT (-96.516486294992674 47.298868727479949)', 'Ada', 0.01, 0.99, 1, 99, 0, 0)
2 ('POINT (-92.71908855924751 43.565362568342763)', 'Adams', 0.02, 0.98, 2, 98, 0, 0)
3 ('POINT (-95.933053872999935 43.633219931412043)', 'Adrian', 0.02, 0.98, 2, 98, 0, 0)
4 ('POINT (-92.817486911940378 44.904199155934208)', 'Afton', 0.7, 0.7, 0, 0, 70, 30)
5 ('POINT (-93.705800484242445 46.526132499502928)', 'Aitkin', 0.02, 0.98, 2, 98, 0, 0)
6 ('POINT (-94.728007078525494 47.000601780541004)', 'Akeley', 0.0, 1.0, 0, 100, 0, 0)
7 ('POINT (-94.567595606746451 45.628320376184469)', 'Albany', 0.11, 0.89, 11, 89, 0, 0)
8 ('POINT (-93.364354198979285 43.654860161036559)', 'Albert Lea', 0.28, 0.72, 28, 72, 0, 0)
9 ('POINT (-96.050377463100844 45.574899738624829)', 'Alberta', 0.0, 1.0, 0, 100, 0, 0)
10 ('POINT (-93.66186685465901 45.236380667012689)', 'Albertville', 0.96, 0.04, 96, 4, 0, 0)
11 ('POINT (-93.573436972396948 43.669520014041005)', 'Alden', 0.0, 1.0, 0, 100

Loading Rank layer to POstGIS

In [9]:
##load Rank Shapefile

#connect to cloud database using psycopg2
#conn = psycopg2.connect(host='34.123.61.225', database='lab0', user='postgres', password='')
conn = psycopg2.connect(host='spatialdb.gisandbox.org', database='marso093', user='marso093', password='')

#create cursor object for interacting with PostGIS
pgcursor = conn.cursor()
print('creating table...')

pgcursor.execute("DROP TABLE IF EXISTS BMSB_City_Rank_alpha2;")

pgcursor.execute("CREATE TABLE BMSB_City_Rank_alpha2(gid serial primary key, shape geometry(POINT, 4326), City_name text, Rank double precision);")

print('Populating Table...')
cur = arcpy.da.SearchCursor('MN_Cities_Pts_WGS_Sort', ('SHAPE@WKT','CITY_SJ','Priority'))
count = 0
for row in cur:
    count +=1
    print(count, row)
    geomwkt = row[0]
    name = row[1]
    rank_values = row[2]
    pgcursor.execute ("INSERT into BMSB_City_Rank_alpha2(shape, City_name, Rank) VALUES(ST_GeomFromText(%s, 4326), %s, %s)", (geomwkt, name, rank_values))

del cur
#commit changes and close connection
conn.commit()
conn.close()


creating table...
Populating Table...
1 ('POINT (-93.230909074394219 44.091294023100716)', 'Owatonna', 1)
2 ('POINT (-93.877075137307713 45.382503098039592)', 'Becker', 2)
3 ('POINT (-93.918452608301209 44.771971992213707)', 'Norwood Young America', 3)
4 ('POINT (-91.668356732477207 44.050593342308957)', 'Winona', 4)
5 ('POINT (-95.057046149260316 45.121687006250397)', 'Willmar', 5)
6 ('POINT (-93.364354199186891 43.654860161267493)', 'Albert Lea', 6)
7 ('POINT (-94.150834044455564 44.769998350843252)', 'Glencoe', 7)
8 ('POINT (-94.198172805630463 46.355270543500808)', 'Brainerd', 8)
9 ('POINT (-94.525503005059591 45.122110538370691)', 'Litchfield', 9)
10 ('POINT (-92.673516302035694 44.294902993978042)', 'Zumbrota', 10)
11 ('POINT (-94.07644181363878 44.608370059530671)', 'Arlington', 11)
12 ('POINT (-95.789692271980755 44.448909565124382)', 'Marshall', 12)
13 ('POINT (-96.745938412427734 46.86747434356613)', 'Moorhead', 13)
14 ('POINT (-94.358783638154364 45.985797249643383)', 'Littl

Loading Accuracy and fake Model 1 output.

## testing our returned geojson Format

In [62]:
#################################################### GeoJson Code return for model 1 : alpha =2 ####################################################
##return point 'difference' layer as geojson

connection = psycopg2.connect(host='spatialdb.gisandbox.org', database='marso093', user='marso093', password='')
#     connection = psycopg2.connect(host='34.123.61.225', database='lab0', user='postgres', password='')
cursor = connection.cursor()
cursor.execute("SELECT json_build_object('type', 'FeatureCollection', 'features', json_agg(features.feature)::jsonb)"
               "FROM (SELECT jsonb_build_object('type', 'Feature', 'geometry', ST_AsGeoJSON(shape)::jsonb, 'properties', jsonb_build_object('gid', gid, 'CITY_SJ', CITY_SJ, 'Presence', Presence, 'Accuracy', Accuracy, 'overall_FP', overall_FP, 'overall_TN', overall_TN, 'overall_TP', overall_TP, 'overall_FN', overall_FN))::jsonb As feature FROM MN_Cities_Pts_WGS_norank) features;")
returns =cursor.fetchall()
connection.close()
returns[0][0]

##took approx. 1.5 mins

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-96.516486295, 47.298868727]}, 'properties': {'gid': 1, 'CITY_SJ': 'Ada', 'Accuracy': 0.99, 'Presence': 0.01, 'overall_FN': 0, 'overall_FP': 1, 'overall_TN': 99, 'overall_TP': 0}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-92.719088559, 43.565362568]}, 'properties': {'gid': 2, 'CITY_SJ': 'Adams', 'Accuracy': 0.98, 'Presence': 0.02, 'overall_FN': 0, 'overall_FP': 2, 'overall_TN': 98, 'overall_TP': 0}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-95.933053873, 43.633219931]}, 'properties': {'gid': 3, 'CITY_SJ': 'Adrian', 'Accuracy': 0.98, 'Presence': 0.02, 'overall_FN': 0, 'overall_FP': 2, 'overall_TN': 98, 'overall_TP': 0}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-92.817486912, 44.904199156]}, 'properties': {'gid': 4, 'CITY_SJ': 'Afton', 'Accuracy': 0.7, 'Presence': 0.7, 'overall_FN': 30, 'overall_FP': 0, 

In [10]:
##return point rank layer as geojson

connection = psycopg2.connect(host='spatialdb.gisandbox.org', database='marso093', user='marso093', password='')
#     connection = psycopg2.connect(host='34.123.61.225', database='lab0', user='postgres', password='')
cursor = connection.cursor()
cursor.execute("SELECT json_build_object('type', 'FeatureCollection', 'features', json_agg(features.feature)::jsonb)"
               "FROM (SELECT jsonb_build_object('type', 'Feature', 'geometry', ST_AsGeoJSON(shape)::jsonb, 'properties', jsonb_build_object('gid', gid, 'City_name', City_name, 'Rank', Rank))::jsonb As feature FROM BMSB_City_Rank_alpha2) features;")
returns =cursor.fetchall()
connection.close()
returns[0][0]

##took approx. 1.5 mins

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-93.230909074, 44.091294023]}, 'properties': {'gid': 1, 'Rank': 1, 'City_name': 'Owatonna'}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-93.877075137, 45.382503098]}, 'properties': {'gid': 2, 'Rank': 2, 'City_name': 'Becker'}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-93.918452608, 44.771971992]}, 'properties': {'gid': 3, 'Rank': 3, 'City_name': 'Norwood Young America'}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-91.668356732, 44.050593342]}, 'properties': {'gid': 4, 'Rank': 4, 'City_name': 'Winona'}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-95.057046149, 45.121687006]}, 'properties': {'gid': 5, 'Rank': 5, 'City_name': 'Willmar'}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-93.364354199, 43.654860161]}, 'properties': {'gid': 6, 'Rank': 6, 'City_name': 'Al